In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib


In [17]:
symbol = 'KO'   # You can change this to any liquid stock
df = yf.Ticker(symbol).history(period='2y')
prices = df['Close'].values.reshape(-1,1)

scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices)

lookback = 60
X, y = [], []
for i in range(lookback, len(prices_scaled)):
    X.append(prices_scaled[i-lookback:i, 0])
    y.append(prices_scaled[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(lookback, 1)),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dropout(0.1),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X, y, epochs=30, batch_size=32, validation_split=0.15, callbacks=[es])

# Save for inference
model.save("bilstm_model.h5")
joblib.dump(scaler, "scaler.save")
print("Training done. Saved bilstm_model.h5 and scaler.save.")


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [ ]:
import tensorflow as tf

def predict_next_close(price_series, model_path="bilstm_model.h5", scaler_path="scaler.save", lookback=60):
    scaler = joblib.load(scaler_path)
    model = tf.keras.models.load_model(model_path)
    prices = price_series.values.reshape(-1, 1)
    prices_scaled = scaler.transform(prices)
    X_pred = prices_scaled[-lookback:]
    X_pred = np.reshape(X_pred, (1, lookback, 1))
    pred_scaled = model.predict(X_pred)
    pred_price = scaler.inverse_transform(pred_scaled)
    return float(pred_price[0,0])


In [ ]:
def simulate_price_paths(last_price, mu, sigma, n_days=10, n_sims=1000):
    dt = 1/252
    paths = np.zeros((n_sims, n_days))
    paths[:,0] = last_price
    for t in range(1, n_days):
        rand = np.random.normal(0, 1, n_sims)
        paths[:,t] = paths[:,t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * rand)
    return paths

def estimate_drift_vol(price_series):
    log_returns = np.log(price_series / price_series.shift(1)).dropna()
    mu = log_returns.mean() * 252
    sigma = log_returns.std() * np.sqrt(252)
    return mu, sigma


In [ ]:
def enhanced_signal(price_history):
    pred_next = predict_next_close(price_history['Close'])
    last_price = price_history['Close'].iloc[-1]
    print(f"BiLSTM predicts next close: {pred_next:.2f}")

    mu, sigma = estimate_drift_vol(price_history['Close'])
    mc_paths = simulate_price_paths(last_price, mu, sigma, n_days=10, n_sims=1000)
    mc_mean = mc_paths[:,-1].mean()
    mc_std = mc_paths[:,-1].std()
    print(f"Monte Carlo forecast 10 days: mean={mc_mean:.2f}, std={mc_std:.2f}")

    is_bullish = (pred_next > last_price) and (mc_mean > last_price)
    return is_bullish, pred_next, mc_mean, mc_std, mc_paths


In [ ]:
symbol = "SPY"  # Change this for other stocks
df = yf.Ticker(symbol).history(period="2y")
result = enhanced_signal(df)
is_bullish, pred_next, mc_mean, mc_std, mc_paths = result
print(f"Bullish: {is_bullish}, BiLSTM: {pred_next:.2f}, MC mean: {mc_mean:.2f}, MC std: {mc_std:.2f}")


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [14]:
# Plot Monte Carlo price simulations
plt.figure(figsize=(12,6))
for i in range(100):
    plt.plot(mc_paths[i], color="skyblue", alpha=0.2)
plt.plot(np.mean(mc_paths, axis=0), color="blue", label="MC Mean Forecast")
plt.axhline(df['Close'].iloc[-1], color="gray", linestyle="--", label="Last Actual Price")
plt.title(f"Monte Carlo Price Simulations for {symbol}")
plt.xlabel("Days into Future")
plt.ylabel("Price")
plt.legend()
plt.show()


NameError: name 'mc_paths' is not defined

<Figure size 1200x600 with 0 Axes>